In [47]:
import numpy as np
from numba import njit, prange
from pokerkit import (
    NoLimitTexasHoldem,
    Automation,
    Card
)
from pokerkit.hands import StandardHighHand  
from collections import Counter
import random

COMBINATIONS_7C5 = np.array([
    [0,1,2,3,4], [0,1,2,3,5], [0,1,2,3,6],
    [0,1,2,4,5], [0,1,2,4,6], [0,1,2,5,6],
    [0,1,3,4,5], [0,1,3,4,6], [0,1,3,5,6],
    [0,1,4,5,6], [0,2,3,4,5], [0,2,3,4,6],
    [0,2,3,5,6], [0,2,4,5,6], [0,3,4,5,6],
    [1,2,3,4,5], [1,2,3,4,6], [1,2,3,5,6],
    [1,2,4,5,6], [1,3,4,5,6], [2,3,4,5,6]
], dtype=np.uint8)

def evaluate_hand(player_hand, board):
    
    hole_cards = "".join(player_hand)  
    board_cards = "".join(board) if board else ""  
    
    hand = StandardHighHand.from_game(hole_cards, board_cards)
    return hand


def calculate_win_percentage(
    player_hand,
    community_cards=None,
    simulations=100000,
    num_decks=1,
    num_opponents=1
):
    community = list(community_cards) if community_cards else []
    needed_community = 5 - len(community)
    known_cards = Counter(player_hand + community)

    deck = [
        f"{r}{s}"
        for _ in range(num_decks)
        for r in '23456789TJQKA'
        for s in 'shdc'
    ]

    wins = 0
    ties = 0

    for _ in range(simulations):
        temp_deck = list(deck)
        for card in known_cards:
            for _ in range(known_cards[card]):
                if card in temp_deck:
                    temp_deck.remove(card)

        random.shuffle(temp_deck)

        
        opp_hands = []
        for _ in range(num_opponents):
            opp_hand = random.sample(temp_deck, 2)
            opp_hands.append(opp_hand)
            for card in opp_hand:
                temp_deck.remove(card)

        
        total_players = num_opponents + 1
        state = NoLimitTexasHoldem.create_state(
            automations=(
                Automation.ANTE_POSTING,
                Automation.BET_COLLECTION,
                Automation.BLIND_OR_STRADDLE_POSTING,
                Automation.HOLE_CARDS_SHOWING_OR_MUCKING,
                Automation.HAND_KILLING,
                Automation.CHIPS_PUSHING,
                Automation.CHIPS_PULLING,
            ),
            raw_starting_stacks=[1000] * total_players,
            player_count=total_players,
            ante_trimming_status=False,
            raw_antes=0,
            raw_blinds_or_straddles=(1, 2),
            min_bet=2
        )

        
        state.deal_hole(tuple(Card.parse(" ".join(player_hand))))

        
        for hand in opp_hands:
            state.deal_hole(tuple(Card.parse(" ".join(hand))))

        
        current_community = list(community)
        while len(current_community) < 5:
            current_community.append(temp_deck.pop())

        
        player_eval = evaluate_hand(player_hand, current_community)
        opponent_evals = [evaluate_hand(opp_hand, current_community) for opp_hand in opp_hands]

        
        all_hands = [player_eval] + opponent_evals
        best_hand = max(all_hands)
        winner_indexes = [i for i, h in enumerate(all_hands) if h == best_hand]

        if 0 in winner_indexes:
            wins += 1 / len(winner_indexes)
        if len(winner_indexes) > 1:
            ties += 1

    return (wins/simulations*100, ties/simulations*100)

def calculate_odds(player_hand, community_cards=None, simulations=100000, num_decks=1, num_opponents=1, use_pokerkit=True):
    if use_pokerkit:
        return calculate_win_percentage(
            player_hand,
            community_cards,
            simulations,
            num_decks,
            num_opponents
        )
    else:
        return (0.0, 0.0)  

if __name__ == "__main__":
    win, tie = calculate_odds(
        ['As', 'Ac'],
        ['Qh', 'Jh', 'Th'],
        simulations=10000,
        num_opponents=3,
        use_pokerkit=True
    )
    print(f"Win: {win:.1f}% | Tie: {tie:.1f}%")


Win: 35.4% | Tie: 6.3%
